In [2]:
import pandas as pd
import numpy
import pyodbc
from utils import *

from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed

from catboost import CatBoostRegressor

C:\Users\bobby\OneDrive\T-drive\Jupyter\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

sql = f'''SELECT 
PTS ,dpm.DID_PLAYER_MISS_GAME as didTeammatePlay

	  ,DATEDIFF(day,lag(pgl.GAME_DATE) over(partition by pgl.PLAYER_ID ORDER BY pgl.game_date),pgl.[GAME_DATE])-1 as daysOfRest
	  ,case when DATEDIFF(day,lag(pgl.GAME_DATE) over(partition by pgl.PLAYER_ID ORDER BY pgl.game_date),pgl.[GAME_DATE])-1 = 0 then 1 else 0 end as secondHalfOfBackToBack

	  ,lag(pgl.PTS) over(partition by pgl.PLAYER_ID, pgl.oppAbrv ORDER BY pgl.game_date) as PtsPlayerGotLastMeeting

	--  ,ROUND(SUM(case when pgl.PTS > @pts_thresh then 1 else 0 end) OVER (PARTITION BY pgl.PLAYER_ID,oppAbrv ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) NumberTimesHitAgaisntOppLast2
	--  ,ROUND(SUM(case when pgl.PTS > @pts_thresh then 1 else 0 end) OVER (PARTITION BY pgl.PLAYER_ID,oppAbrv ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) NumberTimesHitAgaisntOppLast5

	  ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID,oppAbrv ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) PtsPerGameAgainstOppLast2
	  ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID,oppAbrv ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) PtsPerGameAgainstOppLast5

	  ,lag(pgl.PTS) OVER(PARTITION BY pgl.PLAYER_ID ORDER BY pgl.game_date) AS PtsPlayerGotLastGame

	--  ,ROUND(SUM(CAST(case when pgl.PTS > @pts_thresh then 1 else 0 end AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING),2) 
	--  /NULLIF(ROUND(count(CAST(case when pgl.PTS > @pts_thresh then 1 else 0 end AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING),2),0) as PctTimesHitLast10
	--  ,ROUND(SUM(CAST(case when pgl.PTS > @pts_thresh then 1 else 0 end AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) 
	 -- /NULLIF(ROUND(count(CAST(case when pgl.PTS > @pts_thresh then 1 else 0 end AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2),0) as PctTimesHitLast5

      ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PTSLast2
	  ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PTSLast8
	  ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PTSLast20
	  ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 82 PRECEDING AND 1 PRECEDING),2) AS PTSPerGameSeason
	  ,CASE WHEN 
		ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as PTSLongTrend


	  ,ROUND(AVG(CAST(pgl.MIN AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS MinutesPerGameLast2
	  ,ROUND(AVG(CAST(pgl.MIN AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS MinutesPerGameLast8
	  ,ROUND(AVG(CAST(pgl.MIN AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS MinutesPerGameLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(pgl.MIN AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(pgl.MIN AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as MinutesTrend
	   
	  ,ROUND(SUM(CAST(pgl.FGM AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(pgl.FGA AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalPctLast2
	  ,ROUND(SUM(CAST(pgl.FGM AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(pgl.FGA AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalPctLast8
	  ,ROUND(SUM(CAST(pgl.FGM AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(pgl.FGA AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalPctLast20

      ,ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalThreePctLast2
	  ,ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalThreePctLast8
	  ,ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalThreePctLast20
	  ,CASE WHEN 
		ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),0),2) > 
		ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) 
	  THEN 1 else 0 end as FieldGoalThreePctTrend
      
	  ,ROUND(AVG(CAST(pgl.PF AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PersonalFoulsLast8
	  ,ROUND(AVG(CAST(pgl.PFD AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PersonalFoulsDrawnLast8
	  ,CASE WHEN 
		ROUND(AVG(CAST(pgl.PFD AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(pgl.PFD AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as PersonalFoulsDrawnTrend

	  ,ROUND(AVG(CAST(pgl.PLUS_MINUS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PlusMinusLast2
	  ,ROUND(AVG(CAST(pgl.PLUS_MINUS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PlusMinusLast8
	  ,ROUND(AVG(CAST(pgl.PLUS_MINUS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PlusMinusLast20

	  ,ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS FantPointsLast2
	  ,ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS FantPointsLast8
	  ,ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS FantPointsLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as FantPointsDrawnTrend

	  ,ROUND(SUM(CAST(pgl.DD2 AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING),2) AS NumberOfDoubleDoublesLast10
	  ,ROUND(SUM(CAST(pgl.TD3 AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING),2) AS NumberOfTripleDoublesLast10

	  ,RANK() OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ) AS GameNumberThisSeason

	  ,ROUND(AVG(CAST(bsa.offensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS OffensiveRatingLast2
	  ,ROUND(AVG(CAST(bsa.offensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS OffensiveRatingLast8
	  ,ROUND(AVG(CAST(bsa.offensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS OffensiveRatingLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(bsa.offensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(bsa.offensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as OffensiveRatingTrend
	   
	  ,ROUND(AVG(CAST(bsa.defensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS DefensiveRatingLast2
	  ,ROUND(AVG(CAST(bsa.defensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS DefensiveRatingLast8
	  ,ROUND(AVG(CAST(bsa.defensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS DefensiveRatingLast20
	  
	  ,ROUND(AVG(CAST(bsa.netRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS NetRatingLast2
	  ,ROUND(AVG(CAST(bsa.netRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS NetRatingLast8
	  ,ROUND(AVG(CAST(bsa.netRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS NetRatingLast20
	  
	  ,ROUND(AVG(CAST(bsa.effectiveFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS EffectiveFGPctLast2
	  ,ROUND(AVG(CAST(bsa.effectiveFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS EffectiveFGPctLast8
	  ,ROUND(AVG(CAST(bsa.effectiveFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS EffectiveFGPctLast20

	  
      ,ROUND(AVG(CAST(bsa.trueShootingPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS TSPctLast2
	  ,ROUND(AVG(CAST(bsa.trueShootingPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS TSPctLast8
	  ,ROUND(AVG(CAST(bsa.trueShootingPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS TSPctLast20

	  
      ,ROUND(AVG(CAST(bsa.usagePercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS UsagePctLast2
	  ,ROUND(AVG(CAST(bsa.usagePercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS UsagePctLast8
	  ,ROUND(AVG(CAST(bsa.usagePercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS UsagePctLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(bsa.usagePercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(bsa.usagePercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as UsagePctTrend
	 

	  ,ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PossessionsLast2
	  ,ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PossessionsLast8
	  ,ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PossessionsLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as PossessionsTrend
	  
	  ,ROUND(AVG(CAST(bsa.PIE AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PIELast2
	  ,ROUND(AVG(CAST(bsa.PIE AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PIELast8
	  ,ROUND(AVG(CAST(bsa.PIE AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PIELast20

	  
	  	  ,ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PTSOffTOLast2
	  ,ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PTSOffTOLast8
	  ,ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PTSOffTOLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as PTSOffTOTrend

	  ,ROUND(AVG(CAST(bsm.pointsSecondChance AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS SecondChancePtsLast2
	  ,ROUND(AVG(CAST(bsm.pointsSecondChance AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS SecondChancePtsLast8
	  ,ROUND(AVG(CAST(bsm.pointsSecondChance AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS SecondChancePtsLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(bsm.pointsSecondChance AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(bsm.pointsSecondChance AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as SecondChancePtsTrend

	  ,ROUND(AVG(CAST(bsm.pointsFastBreak AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS FastBreakPtsLast2
	  ,ROUND(AVG(CAST(bsm.pointsFastBreak AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS FastBreakPtsLast8
	  ,ROUND(AVG(CAST(bsm.pointsFastBreak AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS FastBreakPtsLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(bsm.pointsFastBreak AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(bsm.pointsFastBreak AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as FastBreakPtsTrend

	  ,ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PaintPtsLast2
	  ,ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PaintPtsLast8
	  ,ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PaintPtsLast20
	  ,CASE WHEN 
		ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
		ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
	  THEN 1 else 0 end as PaintPtsTrend


	  ,ROUND(SUM(CAST(bst.contestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(bst.contestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),0),2) AS ContestedFGPctLast8
	  ,ROUND(SUM(CAST(bst.contestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(bst.contestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) AS ContestedFGPctLast20
	  ,CASE WHEN 
		ROUND(SUM(CAST(bst.contestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(bst.contestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),0),2) > 
		ROUND(SUM(CAST(bst.contestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(bst.contestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) 
	  THEN 1 else 0 end as ContestedFGPctTrend

	  ,ROUND(SUM(CAST(bst.uncontestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(bst.uncontestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),0),2) AS UncontestedFGPctLast2
	  ,ROUND(SUM(CAST(bst.uncontestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(bst.uncontestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),0),2) AS UncontestedFGPctLast8
	  ,ROUND(SUM(CAST(bst.uncontestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
	  NULLIF(SUM(CAST(bst.uncontestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) AS UncontestedFGPctLast20



  FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl

  LEFT OUTER JOIN [nba_game_data].[dbo].[BoxScoreAdvancedV3] bsa
  ON
  cast(pgl.PLAYER_ID as int) = cast(bsa.personId as int)
  AND pgl.GAME_ID = bsa.GAME_ID

   LEFT OUTER JOIN [nba_game_data].[dbo].BoxScoreMiscV3 bsm
   ON
   cast(pgl.PLAYER_ID as int) = cast(bsm.personId as int)
   AND pgl.GAME_ID = bsm.gameId

   LEFT OUTER JOIN [nba_game_data].[dbo].[BoxScorePlayerTrackV3] bst
   ON
   cast(pgl.PLAYER_ID as int) = cast(bst.personId as int)
   AND pgl.GAME_ID = bst.gameId

   LEFT OUTER JOIN [nba_game_data].[dbo].[InfluentialPlayersV1] ifp
   ON
   cast(pgl.PLAYER_ID as int) = cast(ifp.[PLAYER_ID] as int)

   LEFT OUTER JOIN [nba_game_data].[dbo].[GamesPlayerMissedV1] dpm
   ON
   cast(ifp.[PTS_TEAMMATE_ID] as int) = cast(dpm.[PLAYER_ID] as int)
   AND pgl.GAME_ID = dpm.GAME_ID
   
   where pgl.yearSeason = 2024
'''
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
data = pd.read_sql(sql,cnxn)


In [5]:
df = data.loc[data['yearSeason']=='2024'].reset_index(drop=True)

In [8]:
y = data['PTS']
X = data.drop(columns=['PTS'])

In [20]:
model = CatBoostRegressor(iterations=200, learning_rate=0.01, depth=5, thread_count=-1)
model.fit(X, y, eval_set=(X, y), use_best_model=True)
model.get_feature_importance()

0:	learn: 9.0209179	test: 9.0209179	best: 9.0209179 (0)	total: 13.5ms	remaining: 2.68s
1:	learn: 8.9749040	test: 8.9749040	best: 8.9749040 (1)	total: 21.5ms	remaining: 2.13s
2:	learn: 8.9298710	test: 8.9298710	best: 8.9298710 (2)	total: 31.4ms	remaining: 2.06s
3:	learn: 8.8853344	test: 8.8853344	best: 8.8853344 (3)	total: 40.5ms	remaining: 1.98s
4:	learn: 8.8407116	test: 8.8407116	best: 8.8407116 (4)	total: 49.1ms	remaining: 1.91s
5:	learn: 8.7980543	test: 8.7980543	best: 8.7980543 (5)	total: 57.2ms	remaining: 1.85s
6:	learn: 8.7564080	test: 8.7564080	best: 8.7564080 (6)	total: 65.5ms	remaining: 1.8s
7:	learn: 8.7151446	test: 8.7151446	best: 8.7151446 (7)	total: 72.9ms	remaining: 1.75s
8:	learn: 8.6738892	test: 8.6738892	best: 8.6738892 (8)	total: 81.4ms	remaining: 1.73s
9:	learn: 8.6337042	test: 8.6337042	best: 8.6337042 (9)	total: 90.7ms	remaining: 1.72s
10:	learn: 8.5945828	test: 8.5945828	best: 8.5945828 (10)	total: 98.9ms	remaining: 1.7s
11:	learn: 8.5555240	test: 8.5555240	best: 

106:	learn: 6.5699521	test: 6.5699521	best: 6.5699521 (106)	total: 824ms	remaining: 717ms
107:	learn: 6.5609612	test: 6.5609612	best: 6.5609612 (107)	total: 833ms	remaining: 710ms
108:	learn: 6.5524570	test: 6.5524570	best: 6.5524570 (108)	total: 840ms	remaining: 701ms
109:	learn: 6.5428840	test: 6.5428840	best: 6.5428840 (109)	total: 849ms	remaining: 694ms
110:	learn: 6.5336252	test: 6.5336252	best: 6.5336252 (110)	total: 856ms	remaining: 687ms
111:	learn: 6.5241823	test: 6.5241823	best: 6.5241823 (111)	total: 871ms	remaining: 685ms
112:	learn: 6.5152624	test: 6.5152624	best: 6.5152624 (112)	total: 880ms	remaining: 678ms
113:	learn: 6.5072004	test: 6.5072004	best: 6.5072004 (113)	total: 889ms	remaining: 670ms
114:	learn: 6.4987687	test: 6.4987687	best: 6.4987687 (114)	total: 897ms	remaining: 663ms
115:	learn: 6.4905534	test: 6.4905534	best: 6.4905534 (115)	total: 904ms	remaining: 655ms
116:	learn: 6.4828246	test: 6.4828246	best: 6.4828246 (116)	total: 914ms	remaining: 648ms
117:	learn

array([7.23873123e+00, 3.56481403e-02, 0.00000000e+00, 4.36321852e-03,
       9.26201765e-03, 9.92196888e-03, 5.68817920e-01, 1.56273303e+00,
       1.58194755e+01, 2.30342310e+01, 2.50957842e+01, 1.68021594e-02,
       2.64853372e+00, 1.96688164e+00, 1.73489859e+00, 0.00000000e+00,
       6.86578686e-03, 1.18709536e-02, 3.15277711e-02, 9.16236642e-02,
       9.05493044e-03, 4.19452093e-02, 1.15167070e-02, 9.51730018e-02,
       2.17380666e+00, 1.61313326e-02, 2.55397953e-01, 2.88800323e-01,
       1.17180337e-01, 1.31768659e+00, 2.18382804e+00, 1.81862417e+00,
       0.00000000e+00, 1.78729291e-01, 1.20366267e-02, 5.95614869e-03,
       3.03302738e-02, 3.90518406e-02, 1.55613785e-02, 0.00000000e+00,
       3.98788079e-02, 4.17454468e-02, 7.80803440e-02, 0.00000000e+00,
       4.68350712e-02, 3.35523407e-02, 1.00120224e-01, 5.02133339e-02,
       1.52165612e-01, 3.13725658e-02, 3.34947048e-02, 5.74410364e-02,
       7.10489190e-01, 1.98435843e+00, 2.90799663e+00, 0.00000000e+00,
      

In [21]:
# Get feature importances
feature_importances = model.get_feature_importance()

# Assuming X is a DataFrame, get feature names
feature_names = X.columns

# Combine feature names and their importances
features_and_importances = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# If you want to put them in one column, you can create a string representation
features_and_importances['Feature_Importance'] = features_and_importances.apply(lambda row: f"{row['Feature']}: {row['Importance']}", axis=1)

features_and_importances.loc[features_and_importances['Importance']==0].sort_values(by='Importance')

,Feature,Importance,Feature_Importance
2,secondHalfOfBackToBack,0.0,secondHalfOfBackToBack: 0.0
15,MinutesTrend,0.0,MinutesTrend: 0.0
32,FantPointsDrawnTrend,0.0,FantPointsDrawnTrend: 0.0
39,OffensiveRatingTrend,0.0,OffensiveRatingTrend: 0.0
43,NetRatingLast2,0.0,NetRatingLast2: 0.0
55,UsagePctTrend,0.0,UsagePctTrend: 0.0
70,SecondChancePtsTrend,0.0,SecondChancePtsTrend: 0.0
74,FastBreakPtsTrend,0.0,FastBreakPtsTrend: 0.0
81,ContestedFGPctTrend,0.0,ContestedFGPctTrend: 0.0


In [9]:
imp = features_and_importances.sort_values('Importance', ascending  = False).reset_index(drop=True)
imp.loc[imp['Feature']=='total_starters_out']#.iloc[180:240]
imp.iloc[0:60]

,Feature,Importance,Feature_Importance
0,PTSLast20,22.555635,PTSLast20: 22.55563533170051
1,PTSLast10,6.036511,PTSLast10: 6.0365107636113144
2,NBA_FANTASY_PTS_RANKLast20,4.606577,NBA_FANTASY_PTS_RANKLast20: 4.60657662773313
3,PTS_RANKLast10,4.343095,PTS_RANKLast10: 4.343094966390102
4,FGM_RANKLast20,3.188315,FGM_RANKLast20: 3.1883152224062603
5,WNBA_FANTASY_PTSLast10,3.157462,WNBA_FANTASY_PTSLast10: 3.1574624064928787
6,PTS_RANKLast2,2.816135,PTS_RANKLast2: 2.816134862273166
7,FGA_RANKLast10,2.676080,FGA_RANKLast10: 2.676079749783378
8,FGA_RANKLast2,2.303817,FGA_RANKLast2: 2.303816837712114
9,uncontestedFieldGoalsAttemptedLast10,2.218399,uncontestedFieldGoalsAttemptedLast10: 2.218398...


In [58]:
df[['PTS','total_starters_out','did_they_win_last_game','daysRest','RunningGamesWonLast10','PTSLast20','NBA_FANTASY_PTSLast10','FGALast20','FGALast10',
    'matchupFieldGoalsAttemptedLast2','daysRest','MIN_RANKLast2','foulsDrawnLast20','PFDLast10',
    'percentageAssisted2ptLast20','partialPossessionsLast2', 'ASTLast2',
    'oppPointsPaintLast2','offensiveReboundPercentageLast10','PLUS_MINUSLast2','pointsSecondChanceLast20',
    'assistRatioLast20','trueShootingPercentageLast10','trueShootingPercentageLast2','distanceLast2',
    'offensiveRatingLast20','matchupMinutesLast20',
    'matchupFieldGoalsAttemptedLast20']].corr()

,PTS,total_starters_out,did_they_win_last_game,daysRest,RunningGamesWonLast10,PTSLast20,NBA_FANTASY_PTSLast10,FGALast20,FGALast10,matchupFieldGoalsAttemptedLast2,...,offensiveReboundPercentageLast10,PLUS_MINUSLast2,pointsSecondChanceLast20,assistRatioLast20,trueShootingPercentageLast10,trueShootingPercentageLast2,distanceLast2,offensiveRatingLast20,matchupMinutesLast20,matchupFieldGoalsAttemptedLast20
PTS,1.000000,0.043191,0.004647,-0.053675,0.087338,0.727507,0.692483,0.719133,0.719263,0.469280,...,-0.088713,0.050685,0.457999,0.056171,0.296664,0.192352,0.537632,0.331895,0.645792,0.527305
total_starters_out,0.043191,1.000000,-0.043015,-0.125863,-0.056640,-0.048251,-0.043051,-0.043496,-0.032675,-0.001363,...,0.020756,-0.052298,-0.011758,-0.032905,-0.029121,-0.001070,0.008908,-0.041501,-0.044091,-0.047034
did_they_win_last_game,0.004647,-0.043015,1.000000,-0.005447,0.410605,0.023123,0.032162,0.004150,0.003937,0.014856,...,0.012119,0.374850,0.016657,0.016226,0.053062,0.066391,-0.001797,0.071731,0.012190,0.016110
daysRest,-0.053675,-0.125863,-0.005447,1.000000,-0.023275,-0.024969,-0.026149,-0.026398,-0.017822,-0.004978,...,-0.006154,-0.001832,-0.026635,-0.014148,-0.027026,-0.010538,-0.011906,-0.025461,-0.037185,-0.032667
RunningGamesWonLast10,0.087338,-0.056640,0.410605,-0.023275,1.000000,0.109810,0.133852,0.069581,0.066162,0.089391,...,0.037723,0.329624,0.086635,0.079126,0.174723,0.097427,0.084128,0.268538,0.114184,0.108950
PTSLast20,0.727507,-0.048251,0.023123,-0.024969,0.109810,1.000000,0.917095,0.977944,0.959542,0.559123,...,-0.124276,0.082372,0.620301,0.077538,0.419665,0.238211,0.653754,0.465902,0.864662,0.698883
NBA_FANTASY_PTSLast10,0.692483,-0.043051,0.032162,-0.026149,0.133852,0.917095,1.000000,0.886843,0.906316,0.674514,...,0.052305,0.109386,0.691813,0.194907,0.490745,0.277937,0.696872,0.489261,0.869202,0.794309
FGALast20,0.719133,-0.043496,0.004150,-0.026398,0.069581,0.977944,0.886843,1.000000,0.978061,0.538799,...,-0.181743,0.050332,0.564579,0.092816,0.337838,0.191196,0.661885,0.433634,0.873835,0.678970
FGALast10,0.719263,-0.032675,0.003937,-0.017822,0.066162,0.959542,0.906316,0.978061,1.000000,0.557678,...,-0.171337,0.050511,0.555584,0.084448,0.353182,0.197624,0.683551,0.424700,0.854534,0.665406
matchupFieldGoalsAttemptedLast2,0.469280,-0.001363,0.014856,-0.004978,0.089391,0.559123,0.674514,0.538799,0.557678,1.000000,...,0.137545,0.071591,0.540849,0.098417,0.414618,0.306358,0.714616,0.393056,0.668196,0.791149


In [20]:
def train_fold(train_index, test_index, X, y, features):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=5, thread_count=-1, verbose=0)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)
    return model.get_feature_importance()
    
def feature_selection(X,y):
    
    # Prepare data for cross-validation
    kf = KFold(n_splits=20, shuffle=True, random_state=42)
    folds = list(kf.split(X))
    features = X.columns
    
    # Aggregate feature importances from all folds
    aggregated_importances = np.zeros(len(features))
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for train_index, test_index in folds:
            futures.append(executor.submit(train_fold, train_index, test_index, X, y, features))
        
        for future in as_completed(futures):
            fold_importances = future.result()
            aggregated_importances += fold_importances
    
    # Average the importances over all folds
    average_importances = aggregated_importances / kf.n_splits
    
    # Create a DataFrame of features and their average importances
    feature_importances_df = pd.DataFrame({'Feature': features, 'Importance': average_importances})
    
    # Sort the DataFrame by importance
    feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)
    
    # Select top N features
    N = round(len(X)*.2)
    top_n_features = feature_importances_df.head(N)['Feature'].tolist()
    
    return top_n_features

In [3]:
def get_player_last_n_data( last_n_games):
    server = 'localhost\SQLEXPRESS'
    database = 'nba_game_data'
    sql = f'''
    
      SELECT 
    [PLAYER_ID]
    ,[GAME_ID]
    ,oppAbrv
    

    ---last x games data ---
    ,AVG(CAST(FGM AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FGMLast{last_n_games}
    ,AVG(CAST(FGA AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FGALast{last_n_games}
    ,AVG(CAST(FG_PCT AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG_PCTLast{last_n_games}
    ,AVG(CAST(FG3M AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3MLast{last_n_games}
    ,AVG(CAST(FG3A AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3ALast{last_n_games}
    ,AVG(CAST(FG3_PCT AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3_PCTLast{last_n_games}
    ,AVG(CAST(FTM AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FTMLast{last_n_games}
    ,AVG(CAST(FTA AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FTALast{last_n_games}
    ,AVG(CAST(FT_PCT AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FT_PCTLast{last_n_games}
    ,AVG(CAST(OREB AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS OREBLast{last_n_games}
    ,AVG(CAST(DREB AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS DREBLast{last_n_games}
    ,AVG(CAST(REB AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS REBLast{last_n_games}
    ,AVG(CAST(AST AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS ASTLast{last_n_games}
    ,AVG(CAST(TOV AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS TOVLast{last_n_games}
    ,AVG(CAST(STL AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS STLLast{last_n_games}
    ,AVG(CAST(BLK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS BLKLast{last_n_games}
    ,AVG(CAST(BLKA AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS BLKALast{last_n_games}
    ,AVG(CAST(PF AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PFLast{last_n_games}
    ,AVG(CAST(PFD AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PFDLast{last_n_games}
    ,AVG(CAST(PTS AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PTSLast{last_n_games}
    ,AVG(CAST(PLUS_MINUS AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PLUS_MINUSLast{last_n_games}
    ,AVG(CAST(NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS NBA_FANTASY_PTSLast{last_n_games}
    ,AVG(CAST(DD2 AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS DD2Last{last_n_games}
    ,AVG(CAST(TD3 AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS TD3Last{last_n_games}
    ,AVG(CAST(WNBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS WNBA_FANTASY_PTSLast{last_n_games}
    ,AVG(CAST(GP_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS GP_RANKLast{last_n_games}
    ,AVG(CAST(W_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS W_RANKLast{last_n_games}
    ,AVG(CAST(L_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS L_RANKLast{last_n_games}
    ,AVG(CAST(W_PCT_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS W_PCT_RANKLast{last_n_games}
    ,AVG(CAST(MIN_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS MIN_RANKLast{last_n_games}
    ,AVG(CAST(FGM_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FGM_RANKLast{last_n_games}
    ,AVG(CAST(FGA_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FGA_RANKLast{last_n_games}
    ,AVG(CAST(FG_PCT_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG_PCT_RANKLast{last_n_games}
    ,AVG(CAST(FG3M_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3M_RANKLast{last_n_games}
    ,AVG(CAST(FG3A_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3A_RANKLast{last_n_games}
    ,AVG(CAST(FG3_PCT_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3_PCT_RANKLast{last_n_games}
    ,AVG(CAST(FTM_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FTM_RANKLast{last_n_games}
    ,AVG(CAST(FTA_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FTA_RANKLast{last_n_games}
    ,AVG(CAST(FT_PCT_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FT_PCT_RANKLast{last_n_games}
    ,AVG(CAST(OREB_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS OREB_RANKLast{last_n_games}
    ,AVG(CAST(DREB_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS DREB_RANKLast{last_n_games}
    ,AVG(CAST(REB_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS REB_RANKLast{last_n_games}
    ,AVG(CAST(AST_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS AST_RANKLast{last_n_games}
    ,AVG(CAST(TOV_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS TOV_RANKLast{last_n_games}
    ,AVG(CAST(STL_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS STL_RANKLast{last_n_games}
    ,AVG(CAST(BLK_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS BLK_RANKLast{last_n_games}
    ,AVG(CAST(BLKA_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS BLKA_RANKLast{last_n_games}
    ,AVG(CAST(PF_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PF_RANKLast{last_n_games}
    ,AVG(CAST(PFD_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PFD_RANKLast{last_n_games}
    ,AVG(CAST(PTS_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PTS_RANKLast{last_n_games}
    ,AVG(CAST(PLUS_MINUS_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PLUS_MINUS_RANKLast{last_n_games}
    ,AVG(CAST(NBA_FANTASY_PTS_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS NBA_FANTASY_PTS_RANKLast{last_n_games}
    ,AVG(CAST(DD2_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS DD2_RANKLast{last_n_games}
    ,AVG(CAST(TD3_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS TD3_RANKLast{last_n_games}
    ,AVG(CAST(WNBA_FANTASY_PTS_RANK AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS WNBA_FANTASY_PTS_RANKLast{last_n_games}
    ,AVG(CAST(estimatedOffensiveRating AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedOffensiveRatingLast{last_n_games}
    ,AVG(CAST(offensiveRating AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS offensiveRatingLast{last_n_games}
    ,AVG(CAST(estimatedDefensiveRating AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedDefensiveRatingLast{last_n_games}
    ,AVG(CAST(defensiveRating AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defensiveRatingLast{last_n_games}
    ,AVG(CAST(estimatedNetRating AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedNetRatingLast{last_n_games}
    ,AVG(CAST(netRating AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS netRatingLast{last_n_games}
    ,AVG(CAST(assistPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS assistPercentageLast{last_n_games}
    ,AVG(CAST(assistToTurnover AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS assistToTurnoverLast{last_n_games}
    ,AVG(CAST(assistRatio AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS assistRatioLast{last_n_games}
    ,AVG(CAST(offensiveReboundPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS offensiveReboundPercentageLast{last_n_games}
    ,AVG(CAST(defensiveReboundPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defensiveReboundPercentageLast{last_n_games}
    ,AVG(CAST(reboundPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS reboundPercentageLast{last_n_games}
    ,AVG(CAST(turnoverRatio AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS turnoverRatioLast{last_n_games}
    ,AVG(CAST(effectiveFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS effectiveFieldGoalPercentageLast{last_n_games}
    ,AVG(CAST(trueShootingPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS trueShootingPercentageLast{last_n_games}
    ,AVG(CAST(usagePercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS usagePercentageLast{last_n_games}
    ,AVG(CAST(estimatedUsagePercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedUsagePercentageLast{last_n_games}
    ,AVG(CAST(estimatedPace AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedPaceLast{last_n_games}
    ,AVG(CAST(pace AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS paceLast{last_n_games}
    ,AVG(CAST(pacePer40 AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pacePer40Last{last_n_games}
    ,AVG(CAST(possessions AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS possessionsLast{last_n_games}
    ,AVG(CAST(PIE AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PIELast{last_n_games}
    ,AVG(CAST(matchupMinutes AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupMinutesLast{last_n_games}
    ,AVG(CAST(partialPossessions AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS partialPossessionsLast{last_n_games}
    ,AVG(CAST(switchesOn AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS switchesOnLast{last_n_games}
    ,AVG(CAST(playerPoints AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS playerPointsLast{last_n_games}
    ,AVG(CAST(defensiveRebounds AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defensiveReboundsLast{last_n_games}
    ,AVG(CAST(matchupAssists AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupAssistsLast{last_n_games}
    ,AVG(CAST(matchupTurnovers AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupTurnoversLast{last_n_games}
    ,AVG(CAST(matchupFieldGoalsMade AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupFieldGoalsMadeLast{last_n_games}
    ,AVG(CAST(matchupFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupFieldGoalsAttemptedLast{last_n_games}
    ,AVG(CAST(matchupFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupFieldGoalPercentageLast{last_n_games}
    ,AVG(CAST(matchupThreePointersMade AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupThreePointersMadeLast{last_n_games}
    ,AVG(CAST(matchupThreePointersAttempted AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupThreePointersAttemptedLast{last_n_games}
    ,AVG(CAST(matchupThreePointerPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS matchupThreePointerPercentageLast{last_n_games}
    ,AVG(CAST(contestedShots AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedShotsLast{last_n_games}
    ,AVG(CAST(contestedShots2pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedShots2ptLast{last_n_games}
    ,AVG(CAST(contestedShots3pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedShots3ptLast{last_n_games}
    ,AVG(CAST(deflections AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS deflectionsLast{last_n_games}
    ,AVG(CAST(chargesDrawn AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS chargesDrawnLast{last_n_games}
    ,AVG(CAST(screenAssists AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS screenAssistsLast{last_n_games}
    ,AVG(CAST(screenAssistPoints AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS screenAssistPointsLast{last_n_games}
    ,AVG(CAST(looseBallsRecoveredOffensive AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS looseBallsRecoveredOffensiveLast{last_n_games}
    ,AVG(CAST(looseBallsRecoveredDefensive AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS looseBallsRecoveredDefensiveLast{last_n_games}
    ,AVG(CAST(looseBallsRecoveredTotal AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS looseBallsRecoveredTotalLast{last_n_games}
    ,AVG(CAST(offensiveBoxOuts AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS offensiveBoxOutsLast{last_n_games}
    ,AVG(CAST(defensiveBoxOuts AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defensiveBoxOutsLast{last_n_games}
    ,AVG(CAST(boxOutPlayerTeamRebounds AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS boxOutPlayerTeamReboundsLast{last_n_games}
    ,AVG(CAST(boxOutPlayerRebounds AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS boxOutPlayerReboundsLast{last_n_games}
    ,AVG(CAST(boxOuts AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS boxOutsLast{last_n_games}
    ,AVG(CAST(pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pointsOffTurnoversLast{last_n_games}
    ,AVG(CAST(pointsSecondChance AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pointsSecondChanceLast{last_n_games}
    ,AVG(CAST(pointsFastBreak AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pointsFastBreakLast{last_n_games}
    ,AVG(CAST(pointsPaint AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pointsPaintLast{last_n_games}
    ,AVG(CAST(oppPointsOffTurnovers AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS oppPointsOffTurnoversLast{last_n_games}
    ,AVG(CAST(oppPointsSecondChance AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS oppPointsSecondChanceLast{last_n_games}
    ,AVG(CAST(oppPointsFastBreak AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS oppPointsFastBreakLast{last_n_games}
    ,AVG(CAST(oppPointsPaint AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS oppPointsPaintLast{last_n_games}
    ,AVG(CAST(blocksAgainst AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS blocksAgainstLast{last_n_games}
    ,AVG(CAST(foulsPersonal AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS foulsPersonalLast{last_n_games}
    ,AVG(CAST(foulsDrawn AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS foulsDrawnLast{last_n_games}
    ,AVG(CAST(percentageFieldGoalsAttempted2pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageFieldGoalsAttempted2ptLast{last_n_games}
    ,AVG(CAST(percentageFieldGoalsAttempted3pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageFieldGoalsAttempted3ptLast{last_n_games}
    ,AVG(CAST(percentagePoints2pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePoints2ptLast{last_n_games}
    ,AVG(CAST(percentagePointsMidrange2pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsMidrange2ptLast{last_n_games}
    ,AVG(CAST(percentagePoints3pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePoints3ptLast{last_n_games}
    ,AVG(CAST(percentagePointsFastBreak AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsFastBreakLast{last_n_games}
    ,AVG(CAST(percentagePointsFreeThrow AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsFreeThrowLast{last_n_games}
    ,AVG(CAST(percentagePointsOffTurnovers AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsOffTurnoversLast{last_n_games}
    ,AVG(CAST(percentagePointsPaint AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsPaintLast{last_n_games}
    ,AVG(CAST(percentageAssisted2pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageAssisted2ptLast{last_n_games}
    ,AVG(CAST(percentageUnassisted2pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageUnassisted2ptLast{last_n_games}
    ,AVG(CAST(percentageAssisted3pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageAssisted3ptLast{last_n_games}
    ,AVG(CAST(percentageUnassisted3pt AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageUnassisted3ptLast{last_n_games}
    ,AVG(CAST(percentageAssistedFGM AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageAssistedFGMLast{last_n_games}
    ,AVG(CAST(percentageUnassistedFGM AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageUnassistedFGMLast{last_n_games}
    ,AVG(CAST(speed AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS speedLast{last_n_games}
    ,AVG(CAST(distance AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS distanceLast{last_n_games}
    ,AVG(CAST(reboundChancesOffensive AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS reboundChancesOffensiveLast{last_n_games}
    ,AVG(CAST(reboundChancesDefensive AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS reboundChancesDefensiveLast{last_n_games}
    ,AVG(CAST(reboundChancesTotal AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS reboundChancesTotalLast{last_n_games}
    ,AVG(CAST(touches AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS touchesLast{last_n_games}
    ,AVG(CAST(secondaryAssists AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS secondaryAssistsLast{last_n_games}
    ,AVG(CAST(freeThrowAssists AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS freeThrowAssistsLast{last_n_games}
    ,AVG(CAST(passes AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS passesLast{last_n_games}
    ,AVG(CAST(assists AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS assistsLast{last_n_games}
    ,AVG(CAST(contestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedFieldGoalsMadeLast{last_n_games}
    ,AVG(CAST(contestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedFieldGoalsAttemptedLast{last_n_games}
    ,AVG(CAST(contestedFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedFieldGoalPercentageLast{last_n_games}
    ,AVG(CAST(uncontestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS uncontestedFieldGoalsMadeLast{last_n_games}
    ,AVG(CAST(uncontestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS uncontestedFieldGoalsAttemptedLast{last_n_games}
    ,AVG(CAST(uncontestedFieldGoalsPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS uncontestedFieldGoalsPercentageLast{last_n_games}
    ,AVG(CAST(fieldGoalPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS fieldGoalPercentageLast{last_n_games}
    ,AVG(CAST(defendedAtRimFieldGoalsMade AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defendedAtRimFieldGoalsMadeLast{last_n_games}
    ,AVG(CAST(defendedAtRimFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defendedAtRimFieldGoalsAttemptedLast{last_n_games}
    ,AVG(CAST(defendedAtRimFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defendedAtRimFieldGoalPercentageLast{last_n_games}


      FROM BasePlayer
      where yearSeason in (2024,2023)
      order by GAME_DATE

    '''
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    df = pd.read_sql(sql,cnxn)
    
    sql = f'''with base as (
    SELECT  
	   lgl.[SEASON_ID] as SEASON_YEAR
      ,lgl.[TEAM_ID]
      ,lgl.[TEAM_ABBREVIATION]
      ,lgl.[TEAM_NAME]
      ,lgl.[GAME_ID]
      ,lgl.[GAME_DATE]
	  ,lgl.[oppAbrv]
	  ,lgl.[yearSeason]
      ,lgl.[MATCHUP]
      ,lgl.[WL]
      ,lgl.[MIN]
      ,lgl.[FGM]
      ,lgl.[FGA]
      ,lgl.[FG_PCT]
      ,lgl.[FG3M]
      ,lgl.[FG3A]
      ,lgl.[FG3_PCT]
      ,lgl.[FTM]
      ,lgl.[FTA]
      ,lgl.[FT_PCT]
      ,lgl.[OREB]
      ,lgl.[DREB]
      ,lgl.[REB]
      ,lgl.[AST]
      ,lgl.[STL]
      ,lgl.[BLK]
      ,lgl.[TOV]
      ,lgl.[PF]
      ,lgl.[PTS]
      ,lgl.[PLUS_MINUS]

	  ,tbsa.[estimatedOffensiveRating]
      ,tbsa.[offensiveRating]
      ,tbsa.[estimatedDefensiveRating]
      ,tbsa.[defensiveRating]
      ,tbsa.[estimatedNetRating]
      ,tbsa.[netRating]
      ,tbsa.[assistPercentage]
      ,tbsa.[assistToTurnover]
      ,tbsa.[assistRatio]
      ,tbsa.[offensiveReboundPercentage]
      ,tbsa.[defensiveReboundPercentage]
      ,tbsa.[reboundPercentage]
      ,tbsa.[estimatedTeamTurnoverPercentage]
      ,tbsa.[turnoverRatio]
      ,tbsa.[effectiveFieldGoalPercentage]
      ,tbsa.[trueShootingPercentage]
      ,tbsa.[usagePercentage]
      ,tbsa.[estimatedUsagePercentage]
      ,tbsa.[estimatedPace]
      ,tbsa.[pace]
      ,tbsa.[pacePer40]
      ,tbsa.[possessions]
      ,tbsa.[PIE]

	  ,tbsh.[contestedShots]
      ,tbsh.[contestedShots2pt]
      ,tbsh.[contestedShots3pt]
      ,tbsh.[deflections]
      ,tbsh.[chargesDrawn]
      ,tbsh.[screenAssists]
      ,tbsh.[screenAssistPoints]
      ,tbsh.[looseBallsRecoveredOffensive]
      ,tbsh.[looseBallsRecoveredDefensive]
      ,tbsh.[looseBallsRecoveredTotal]
      ,tbsh.[offensiveBoxOuts]
      ,tbsh.[defensiveBoxOuts]
      ,tbsh.[boxOutPlayerTeamRebounds]
      ,tbsh.[boxOutPlayerRebounds]
      ,tbsh.[boxOuts]

	  ,tbsm.[pointsOffTurnovers]
      ,tbsm.[pointsSecondChance]
      ,tbsm.[pointsFastBreak]
      ,tbsm.[pointsPaint]
      ,tbsm.[oppPointsOffTurnovers]
      ,tbsm.[oppPointsSecondChance]
      ,tbsm.[oppPointsFastBreak]
      ,tbsm.[oppPointsPaint]
      ,tbsm.[blocks]
      ,tbsm.[blocksAgainst]
      ,tbsm.[foulsPersonal]
      ,tbsm.[foulsDrawn]

	  ,tbst.[distance]
      ,tbst.[reboundChancesOffensive]
      ,tbst.[reboundChancesDefensive]
      ,tbst.[reboundChancesTotal]
      ,tbst.[touches]
      ,tbst.[secondaryAssists]
      ,tbst.[freeThrowAssists]
      ,tbst.[passes]
      ,tbst.[assists]
      ,tbst.[contestedFieldGoalsMade]
      ,tbst.[contestedFieldGoalsAttempted]
      ,tbst.[contestedFieldGoalPercentage]
      ,tbst.[uncontestedFieldGoalsMade]
      ,tbst.[uncontestedFieldGoalsAttempted]
      ,tbst.[uncontestedFieldGoalsPercentage]
      ,tbst.[fieldGoalPercentage]
      ,tbst.[defendedAtRimFieldGoalsMade]
      ,tbst.[defendedAtRimFieldGoalsAttempted]
      ,tbst.[defendedAtRimFieldGoalPercentage]

      ,tbss.[percentageFieldGoalsAttempted2pt]
      ,tbss.[percentageFieldGoalsAttempted3pt]
      ,tbss.[percentagePoints2pt]
      ,tbss.[percentagePointsMidrange2pt]
      ,tbss.[percentagePoints3pt]
      ,tbss.[percentagePointsFastBreak]
      ,tbss.[percentagePointsFreeThrow]
      ,tbss.[percentagePointsOffTurnovers]
      ,tbss.[percentagePointsPaint]
      ,tbss.[percentageAssisted2pt]
      ,tbss.[percentageUnassisted2pt]
      ,tbss.[percentageAssisted3pt]
      ,tbss.[percentageUnassisted3pt]
      ,tbss.[percentageAssistedFGM]
      ,tbss.[percentageUnassistedFGM]

  FROM [nba_game_data].[dbo].[LeagueGameLog] lgl 

  LEFT OUTER JOIN [nba_game_data].[dbo].[TeamBoxScoreAdvancedV3] tbsa
  on lgl.GAME_ID = cast(tbsa.gameId as int)
  and lgl.TEAM_ID = tbsa.teamId

    LEFT OUTER JOIN [nba_game_data].[dbo].[TeamBoxScoreHustleV2] tbsh
  on lgl.GAME_ID = cast(tbsh.gameId as int)
  and lgl.TEAM_ID = tbsh.teamId

    LEFT OUTER JOIN [nba_game_data].[dbo].[TeamBoxScoreMiscV3] tbsm
  on lgl.GAME_ID = cast(tbsm.gameId as int)
  and lgl.TEAM_ID = tbsm.teamId

  LEFT OUTER JOIN [nba_game_data].[dbo].[TeamBoxScorePlayerTrackV3] tbst
  on lgl.GAME_ID = cast(tbst.gameId as int)
  and lgl.TEAM_ID = tbst.teamId

  LEFT OUTER JOIN [nba_game_data].[dbo].[TeamBoxScoreScoringV3] tbss
  on lgl.GAME_ID = cast(tbss.gameId as int)
  and lgl.TEAM_ID = tbss.teamId
  )
  select 
	   [TEAM_ABBREVIATION]
      ,[GAME_ID]
	  ,AVG(CAST(FGM AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FGMTeamLast{last_n_games}
	  ,AVG(CAST(FGA AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FGATeamLast{last_n_games}
	  ,AVG(CAST(FG_PCT AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG_PCTTeamLast{last_n_games}
	  ,AVG(CAST(FG3M AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3MTeamLast{last_n_games}
	  ,AVG(CAST(FG3A AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3ATeamLast{last_n_games}
	  ,AVG(CAST(FG3_PCT AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FG3_PCTTeamLast{last_n_games}
	  ,AVG(CAST(FTM AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FTMTeamLast{last_n_games}
	  ,AVG(CAST(FTA AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FTATeamLast{last_n_games}
	  ,AVG(CAST(FT_PCT AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS FT_PCTTeamLast{last_n_games}
	  ,AVG(CAST(OREB AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS OREBTeamLast{last_n_games}
	  ,AVG(CAST(DREB AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS DREBTeamLast{last_n_games}
	  ,AVG(CAST(REB AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS REBTeamLast{last_n_games}
	  ,AVG(CAST(AST AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS ASTTeamLast{last_n_games}
	  ,AVG(CAST(STL AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS STLTeamLast{last_n_games}
	  ,AVG(CAST(BLK AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS BLKTeamLast{last_n_games}
	  ,AVG(CAST(TOV AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS TOVTeamLast{last_n_games}
	  ,AVG(CAST(PF AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PFTeamLast{last_n_games}
	  ,AVG(CAST(PTS AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PTSTeamLast{last_n_games}
	  ,AVG(CAST(PLUS_MINUS AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PLUS_MINUSTeamLast{last_n_games}
	  ,AVG(CAST(estimatedOffensiveRating AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedOffensiveRatingTeamLast{last_n_games}
	  ,AVG(CAST(offensiveRating AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS offensiveRatingTeamLast{last_n_games}
	  ,AVG(CAST(estimatedDefensiveRating AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedDefensiveRatingTeamLast{last_n_games}
	  ,AVG(CAST(defensiveRating AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defensiveRatingTeamLast{last_n_games}
	  ,AVG(CAST(estimatedNetRating AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedNetRatingTeamLast{last_n_games}
	  ,AVG(CAST(netRating AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS netRatingTeamLast{last_n_games}
	  ,AVG(CAST(assistPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS assistPercentageTeamLast{last_n_games}
	  ,AVG(CAST(assistToTurnover AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS assistToTurnoverTeamLast{last_n_games}
	  ,AVG(CAST(assistRatio AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS assistRatioTeamLast{last_n_games}
	  ,AVG(CAST(offensiveReboundPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS offensiveReboundPercentageTeamLast{last_n_games}
	  ,AVG(CAST(defensiveReboundPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defensiveReboundPercentageTeamLast{last_n_games}
	  ,AVG(CAST(reboundPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS reboundPercentageTeamLast{last_n_games}
	  ,AVG(CAST(estimatedTeamTurnoverPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedTeamTurnoverPercentageTeamLast{last_n_games}
	  ,AVG(CAST(turnoverRatio AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS turnoverRatioTeamLast{last_n_games}
	  ,AVG(CAST(effectiveFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS effectiveFieldGoalPercentageTeamLast{last_n_games}
	  ,AVG(CAST(trueShootingPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS trueShootingPercentageTeamLast{last_n_games}
	  ,AVG(CAST(usagePercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS usagePercentageTeamLast{last_n_games}
	  ,AVG(CAST(estimatedUsagePercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedUsagePercentageTeamLast{last_n_games}
	  ,AVG(CAST(estimatedPace AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS estimatedPaceTeamLast{last_n_games}
	  ,AVG(CAST(pace AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS paceTeamLast{last_n_games}
	  ,AVG(CAST(pacePer40 AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pacePer40TeamLast{last_n_games}
	  ,AVG(CAST(possessions AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS possessionsTeamLast{last_n_games}
	  ,AVG(CAST(PIE AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS PIETeamLast{last_n_games}
	  ,AVG(CAST(contestedShots AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedShotsTeamLast{last_n_games}
	  ,AVG(CAST(contestedShots2pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedShots2ptTeamLast{last_n_games}
	  ,AVG(CAST(contestedShots3pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedShots3ptTeamLast{last_n_games}
	  ,AVG(CAST(deflections AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS deflectionsTeamLast{last_n_games}
	  ,AVG(CAST(chargesDrawn AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS chargesDrawnTeamLast{last_n_games}
	  ,AVG(CAST(screenAssists AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS screenAssistsTeamLast{last_n_games}
	  ,AVG(CAST(screenAssistPoints AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS screenAssistPointsTeamLast{last_n_games}
	  ,AVG(CAST(looseBallsRecoveredOffensive AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS looseBallsRecoveredOffensiveTeamLast{last_n_games}
	  ,AVG(CAST(looseBallsRecoveredDefensive AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS looseBallsRecoveredDefensiveTeamLast{last_n_games}
	  ,AVG(CAST(looseBallsRecoveredTotal AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS looseBallsRecoveredTotalTeamLast{last_n_games}
	  ,AVG(CAST(offensiveBoxOuts AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS offensiveBoxOutsTeamLast{last_n_games}
	  ,AVG(CAST(defensiveBoxOuts AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defensiveBoxOutsTeamLast{last_n_games}
	  ,AVG(CAST(boxOutPlayerTeamRebounds AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS boxOutPlayerTeamReboundsTeamLast{last_n_games}
	  ,AVG(CAST(boxOutPlayerRebounds AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS boxOutPlayerReboundsTeamLast{last_n_games}
	  ,AVG(CAST(boxOuts AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS boxOutsTeamLast{last_n_games}
	  ,AVG(CAST(pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pointsOffTurnoversTeamLast{last_n_games}
	  ,AVG(CAST(pointsSecondChance AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pointsSecondChanceTeamLast{last_n_games}
	  ,AVG(CAST(pointsFastBreak AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pointsFastBreakTeamLast{last_n_games}
	  ,AVG(CAST(pointsPaint AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS pointsPaintTeamLast{last_n_games}
	  ,AVG(CAST(oppPointsOffTurnovers AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS oppPointsOffTurnoversTeamLast{last_n_games}
	  ,AVG(CAST(oppPointsSecondChance AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS oppPointsSecondChanceTeamLast{last_n_games}
	  ,AVG(CAST(oppPointsFastBreak AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS oppPointsFastBreakTeamLast{last_n_games}
	  ,AVG(CAST(oppPointsPaint AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS oppPointsPaintTeamLast{last_n_games}
	  ,AVG(CAST(blocks AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS blocksTeamLast{last_n_games}
	  ,AVG(CAST(blocksAgainst AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS blocksAgainstTeamLast{last_n_games}
	  ,AVG(CAST(foulsPersonal AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS foulsPersonalTeamLast{last_n_games}
	  ,AVG(CAST(foulsDrawn AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS foulsDrawnTeamLast{last_n_games}
	  ,AVG(CAST(distance AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS distanceTeamLast{last_n_games}
	  ,AVG(CAST(reboundChancesOffensive AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS reboundChancesOffensiveTeamLast{last_n_games}
	  ,AVG(CAST(reboundChancesDefensive AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS reboundChancesDefensiveTeamLast{last_n_games}
	  ,AVG(CAST(reboundChancesTotal AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS reboundChancesTotalTeamLast{last_n_games}
	  ,AVG(CAST(touches AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS touchesTeamLast{last_n_games}
	  ,AVG(CAST(secondaryAssists AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS secondaryAssistsTeamLast{last_n_games}
	  ,AVG(CAST(freeThrowAssists AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS freeThrowAssistsTeamLast{last_n_games}
	  ,AVG(CAST(passes AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS passesTeamLast{last_n_games}
	  ,AVG(CAST(assists AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS assistsTeamLast{last_n_games}
	  ,AVG(CAST(contestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedFieldGoalsMadeTeamLast{last_n_games}
	  ,AVG(CAST(contestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedFieldGoalsAttemptedTeamLast{last_n_games}
	  ,AVG(CAST(contestedFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS contestedFieldGoalPercentageTeamLast{last_n_games}
	  ,AVG(CAST(uncontestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS uncontestedFieldGoalsMadeTeamLast{last_n_games}
	  ,AVG(CAST(uncontestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS uncontestedFieldGoalsAttemptedTeamLast{last_n_games}
	  ,AVG(CAST(uncontestedFieldGoalsPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS uncontestedFieldGoalsPercentageTeamLast{last_n_games}
	  ,AVG(CAST(fieldGoalPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS fieldGoalPercentageTeamLast{last_n_games}
	  ,AVG(CAST(defendedAtRimFieldGoalsMade AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defendedAtRimFieldGoalsMadeTeamLast{last_n_games}
	  ,AVG(CAST(defendedAtRimFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defendedAtRimFieldGoalsAttemptedTeamLast{last_n_games}
	  ,AVG(CAST(defendedAtRimFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS defendedAtRimFieldGoalPercentageTeamLast{last_n_games}
	  ,AVG(CAST(percentageFieldGoalsAttempted2pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageFieldGoalsAttempted2ptTeamLast{last_n_games}
	  ,AVG(CAST(percentageFieldGoalsAttempted3pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageFieldGoalsAttempted3ptTeamLast{last_n_games}
	  ,AVG(CAST(percentagePoints2pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePoints2ptTeamLast{last_n_games}
	  ,AVG(CAST(percentagePointsMidrange2pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsMidrange2ptTeamLast{last_n_games}
	  ,AVG(CAST(percentagePoints3pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePoints3ptTeamLast{last_n_games}
	  ,AVG(CAST(percentagePointsFastBreak AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsFastBreakTeamLast{last_n_games}
	  ,AVG(CAST(percentagePointsFreeThrow AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsFreeThrowTeamLast{last_n_games}
	  ,AVG(CAST(percentagePointsOffTurnovers AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsOffTurnoversTeamLast{last_n_games}
	  ,AVG(CAST(percentagePointsPaint AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentagePointsPaintTeamLast{last_n_games}
	  ,AVG(CAST(percentageAssisted2pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageAssisted2ptTeamLast{last_n_games}
	  ,AVG(CAST(percentageUnassisted2pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageUnassisted2ptTeamLast{last_n_games}
	  ,AVG(CAST(percentageAssisted3pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageAssisted3ptTeamLast{last_n_games}
	  ,AVG(CAST(percentageUnassisted3pt AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageUnassisted3ptTeamLast{last_n_games}
	  ,AVG(CAST(percentageAssistedFGM AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageAssistedFGMTeamLast{last_n_games}
	  ,AVG(CAST(percentageUnassistedFGM AS FLOAT)) OVER (PARTITION BY TEAM_ID,  SEASON_YEAR  ORDER BY GAME_DATE  ROWS BETWEEN {last_n_games} PRECEDING AND 1 PRECEDING) AS percentageUnassistedFGMTeamLast{last_n_games}

      FROM base
      where yearSeason in (2024,2023)
      '''
    team = pd.read_sql(sql,cnxn)

    return df.merge(team, left_on=['oppAbrv','GAME_ID'], right_on=['TEAM_ABBREVIATION','GAME_ID'])